In [ ]:
import requests
import json
import math
import os
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [ ]:
#Dados para o login
login = ""
senha = ""

In [ ]:
defaultUrl = 'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
operations = {'login': 'login',
              'logout' : 'logout',
              'datasetfields' : 'datasetfields',
              'search' : 'search',
              'download' : 'download'
             }


In [ ]:
def login(usr,pwd, catalog = 'EE'):
    payload = {"jsonRequest" : '{"username":"' + usr + '","password":"' + pwd + '","catalogId":"' + catalog + '"}'}
    response = requests.post(defaultUrl + operations['login'],data = payload)
    if response.json()['errorCode'] is None:
        return response.json()['data']
    else:
        print("Login inválido!")
        return None

In [ ]:
def calculateMBR(point,radius):
    coef = radius * 0.0000089;
    ll = (point[0] - coef,point[1] - coef / math.cos(point[0] * 0.018))
    ur = (point[0] + coef,point[1] + coef / math.cos(point[0] * 0.018))
    return ll, ur

In [ ]:
def sceneSearch(apiKey,point,radius,startDate,endDate,
                dataSetName = "SENTINEL_2A", months = None,
                includeUnknownCloudCover = True,maxResults = 1000, sortOrder = 'ASC'):
    ll, ur = calculateMBR(point,radius)
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + dataSetName + '","spatialFilter":{"filterType":"mbr","lowerLeft":{"latitude":"' + str(ll[0]) + '","longitude":"' + str(ll[1]) + '"},"upperRight":{"latitude":"' + str(ur[0]) + '","longitude":"' + str(ur[1]) + '"}},"temporalFilter":{"startDate":"' + startDate + '","endDate":"' + endDate + '"},'+( '' if months is None else '"months":' + str(months) + ',') + '"includeUnknownCloudCover":' + str(includeUnknownCloudCover).lower() + ',"maxResults":"' + str(maxResults) + '","sortOrder":"' + sortOrder + '"}'}
    response = requests.post(defaultUrl + operations['search'], data = payload)
    return response.json()

In [ ]:
def getScenesDict():
    if not os.path.isfile('scenes.json'):
        return json.loads('{}')
    else:
        with open('scenes.json','r') as jsonFile:
            return json.load(jsonFile)

In [ ]:
def downloadFile(downloadInfo,apiKey):
    print(downloadInfo[1])
    payload = '{"apiKey": "' + apiKey +'","datasetName":"SENTINEL_2A","products":"' + downloadInfo[1] + '","entityIds":"' + downloadInfo[2] + '"}'
    print(json.dumps(payload))
    #magicUrl = 'https://earthexplorer.usgs.gov/download/10880/%s/STANDARD/' % downloadInfo[2]
    magicUrl = defaultUrl +  operations['download']
    print(magicUrl)
    with requests.get(magicUrl, data=payload, stream=True, allow_redirects=True) as request:
        #lança exceção caso request de errado
        request.raise_for_status()
        with open(downloadInfo[0] + '.zip', 'wb') as file:
            for chunk in request.iter_content(chunk_size=1024): 
                file.write(chunk)
        print(response.text)
    return True;

In [ ]:
def countDown(path):
    dNumber = 0
    for fname in os.listdir(path):
        if fname.endswith('.crdownload'):
            dNumber +=1
    return dNumber

In [ ]:
def downloadBulk(lat,lon,radius,startDate,endDate,usr,psw,maxDownload = 1, driverPath = '/home/adrianoii/Repo/DesmateSeletivo/chromedriver',downloadPath ='/home/adrianoii/Repo/DesmateSeletivo'):
    apiKey = login(usr,psw)
    scenes = sceneSearch(apiKey,(lat,lon),radius,startDate,endDate)
    scenesJsonDict = getScenesDict()
    downloadList = []
    pointStr = f'({str(lat)},{str(lon)})'
    for foundScene in scenes['data']['results']:
        if foundScene['displayId'] in scenesJsonDict.keys():
            print("Imagem já baixada!")
            if pointStr not in scenesJsonDict[foundScene['displayId']]['points']:
                print("Novo ponto!")
                scenesJsonDict[foundScene['displayId']]['points'].append(pointStr)
        else:
            downloadList.append((foundScene['displayId'],foundScene['downloadUrl'],foundScene['entityId']))
    chromeOptions = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : downloadPath}
    chromeOptions.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(executable_path= driverPath, options=chromeOptions)
    downloadsQueue = []
    for download in downloadList:
        numActualDownloads = countDown(downloadPath)
        numBeforeDownloads =  numActualDownloads
        if(numActualDownloads >= maxDownload):
            print(countDown(downloadPath),maxDownload)
            while ((numActualDownloads := countDown(downloadPath)) >= maxDownload):
                time.sleep(10)
            for downloadFile in downloadsQueue:
                if os.path.isfile(downloadFile["displayId"] + ".zip"):
                    downloadsQueue.remove(downloadFile)
                else:
                    clickToDownload(usr,psw,downloadFile["downloadUrl"],driver)
        clickToDownload(usr,psw,download[1],driver)
        downloadsQueue.append({ "displayId" : download[0],
                                "downloadUrl" : download[1]})
        scenesJsonDict[download[0]] = {"points" : [pointStr],
                                           "startDate" : startDate,
                                           "endDate" : endDate,
                                           "downloadUrl" : download[1]}
    while len(downloadsQueue) > 0:
        for downloadFile in downloadsQueue:
            if os.path.isfile(downloadFile["displayId"] + ".zip"):
                downloadsQueue.remove(downloadFile)
            else:
                clickToDownload(usr,psw,downloadFile["downloadUrl"],driver)
            time.sleep(30)
    driver.close()
    with open('scenes.json','w+') as jsonFile:
        json.dump(scenesJsonDict,jsonFile)

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2017-01-01',login,senha)

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2016-02-01',login,senha)

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2016-01-07',login,senha)

In [ ]:
def clickToDownload(usr,pwd,link,driver):
    driver.get(link)
    try:
        login = driver.find_element_by_id('loginLink')
    except NoSuchElementException:
        print("Logado")
    else:
        login.click()
        driver.find_element_by_id('username').send_keys(usr)
        driver.find_element_by_id('password').send_keys(pwd)
        driver.find_element_by_id('loginButton').submit()
    finally:
        driver.find_element_by_xpath(".//*[@title='Download this Product']").click()

In [ ]:
clickToDownload(login,senha
                
                ,"https://earthexplorer.usgs.gov/download/external/options/SENTINEL_2A/1100852/INVSVC/")